In [1]:
# imports
import sys
import os
import socket
import pandas as pd
import glob

In [2]:
# function to format dates to datetime and replace nans in eventdate with sysdate
def formatdate(data):
    data['sysdate'] = data['sysdate'].str.replace('/', '')
    data['sysdate']= pd.to_datetime(data['sysdate'], format='%d%m%Y')
    data['eventdate']= data['eventdate'].str.replace('/', '')
    data['eventdate'] = pd.to_datetime(data['eventdate'], format='%d%m%Y')
    data['eventdate'].fillna(data['sysdate'], inplace = True)
    return data

In [3]:
next(os.walk('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD'))

('/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD',
 [],
 ['19_050_gold_extract_therapy_072.txt',
  '19_050_gold_extract_test_010.txt',
  '19_050_gold_extract_therapy_055.txt',
  '19_050_gold_extract_therapy_113.txt',
  '19_050_gold_extract_therapy_033.txt',
  '19_050_gold_extract_therapy_077.txt',
  '19_050_gold_extract_therapy_023.txt',
  '19_050_gold_extract_clinical_028.txt',
  '19_050_gold_extract_test_018.txt',
  '19_050_gold_extract_test_027.txt',
  '19_050_gold_extract_clinical_010.txt',
  '19_050_gold_extract_test_012.txt',
  '19_050_gold_extract_additional_001.txt',
  '19_050_gold_extract_test_023.txt',
  '19_050_gold_extract_therapy_036.txt',
  '19_050_gold_extract_therapy_062.txt',
  '19_050_gold_extract_clinical_024.txt',
  '19_050_gold_extract_therapy_007.txt',
  '19_050_gold_extract_therapy_079.txt',
  '19_050_gold_extract_therapy_060.txt',
  '19_050_gold_extract_therapy_099.txt',
  '19_050_gold_extract_therapy_061.txt',
  '19_050_gold_extr

In [4]:
path = '/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD/'

In [5]:
# read in dementia codes
dementia_codes = pd.read_excel('/mnt/maths/tws21/dementia_codes.xlsx')

In [6]:
dementia_codes.head()

,medcode,readcode,readterm,type,probable,possible,diagnosis,history,admin,assessment,suspected,referral,drugs
0,1917,F110.00,Alzheimer's disease,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,7664,Eu00.00,[X]Dementia in Alzheimer's disease,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,8195,Eu00z11,[X]Alzheimer's dementia unspec,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,11379,Eu00112,"[X]Senile dementia,Alzheimer's type",Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,16797,F110000,Alzheimer's disease with early onset,Alzheimer's,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# select only diagnosis med codes
dementia_med_codes = dementia_codes[dementia_codes['diagnosis'] == 1.0]['medcode']

In [8]:
dementia_med_codes

0       1917
1       7664
2       8195
3      11379
4      16797
       ...  
136    49513
137    53446
138    55222
139    55838
153    30706
Name: medcode, Length: 94, dtype: int64

In [9]:
#read in and merge all the test files and search for the dementia diagnosis medcodes

test_files = glob.glob(path + "*test*.txt")

test_list = []
for filename in test_files:
    df = pd.read_csv(filename, sep="\t", dtype = ({'data8':'str'}))
    df = df[df['medcode'].isin(list(dementia_med_codes))]
    test_list.append(df)

test_merged = pd.concat(test_list, axis=0)

In [10]:
test_merged.head()

,e_patid,eventdate,sysdate,constype,consid,medcode,enttype,data1,data2,data3,data4,data5,data6,data7,data8
506109,157719246,19/07/2001,07/02/2002,2,1115667,1350,288,0.0,NaN,0.0,0,NaN,NaN,0.0,NaN
506110,157719246,19/07/2001,07/02/2002,2,1115667,1350,288,0.0,NaN,0.0,0,NaN,NaN,0.0,NaN
506115,157719246,19/07/2001,07/02/2002,2,1115667,1350,288,0.0,NaN,0.0,0,NaN,NaN,0.0,NaN
506116,157719246,19/07/2001,07/02/2002,2,1115667,1350,288,0.0,NaN,0.0,0,NaN,NaN,0.0,NaN
506118,157719246,19/07/2001,07/02/2002,2,1115667,1350,288,0.0,NaN,0.0,0,NaN,NaN,0.0,NaN


In [11]:
# format date
test_merged = formatdate(test_merged)

In [12]:
# save as parquet
test_merged.to_parquet(os.getcwd() + '/dementia_patients_test_data.parquet')